# 25 persons
## electrodes : 8, 16, 32, 64

In [18]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier

In [4]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [19]:
no_of_patients=25

In [20]:
train=sorted(glob('files2/*.edf'))
train=train[:no_of_patients]
train

['files2/S001R04.edf',
 'files2/S002R04.edf',
 'files2/S003R04.edf',
 'files2/S004R04.edf',
 'files2/S005R04.edf',
 'files2/S006R04.edf',
 'files2/S007R04.edf',
 'files2/S008R04.edf',
 'files2/S009R04.edf',
 'files2/S010R04.edf',
 'files2/S011R04.edf',
 'files2/S012R04.edf',
 'files2/S013R04.edf',
 'files2/S014R04.edf',
 'files2/S015R04.edf',
 'files2/S016R04.edf',
 'files2/S017R04.edf',
 'files2/S018R04.edf',
 'files2/S019R04.edf',
 'files2/S020R04.edf',
 'files2/S021R04.edf',
 'files2/S022R04.edf',
 'files2/S023R04.edf',
 'files2/S024R04.edf',
 'files2/S025R04.edf']

In [21]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [22]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [23]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [24]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [25]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(300000, 37475, 300000, 37475)

In [26]:
print(xtest.iloc[:,-1].values)

[-1.0e-05  1.5e-05  6.0e-06 ...  4.0e-06  1.0e-06  3.0e-06]


In [27]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.000000,0.000008,0.000001,0.000003,-0.000005,0.000001,-0.000003,-0.000008,-0.000004,0.000007,...,-0.000009,0.000000,0.000009,0.000017,0.000005,0.000002,0.000015,0.000003,0.000004,0.000012
299996,-0.000006,0.000014,0.000009,0.000012,0.000005,0.000010,0.000007,-0.000002,-0.000001,0.000012,...,-0.000005,0.000006,0.000013,0.000018,0.000007,0.000012,0.000015,0.000009,0.000012,0.000017
299997,0.000008,0.000017,0.000016,0.000021,0.000009,0.000010,0.000005,-0.000002,-0.000002,0.000014,...,-0.000012,-0.000003,0.000002,0.000004,-0.000008,-0.000005,0.000005,-0.000011,-0.000010,-0.000004
299998,-0.000017,-0.000003,-0.000001,0.000004,-0.000005,-0.000004,-0.000006,-0.000018,-0.000015,0.000002,...,-0.000010,-0.000004,0.000000,0.000004,-0.000007,-0.000002,0.000004,-0.000011,-0.000009,-0.000003


In [28]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [29]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [30]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_10231/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_10231/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_10231/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.000000,0.000008,0.000001,0.000003,-0.000005,0.000001,-0.000003,-0.000008,-0.000004,0.000007,...,-0.000009,0.000000,0.000009,0.000017,0.000005,0.000002,0.000015,0.000003,0.000004,0.000012
299996,-0.000006,0.000014,0.000009,0.000012,0.000005,0.000010,0.000007,-0.000002,-0.000001,0.000012,...,-0.000005,0.000006,0.000013,0.000018,0.000007,0.000012,0.000015,0.000009,0.000012,0.000017
299997,0.000008,0.000017,0.000016,0.000021,0.000009,0.000010,0.000005,-0.000002,-0.000002,0.000014,...,-0.000012,-0.000003,0.000002,0.000004,-0.000008,-0.000005,0.000005,-0.000011,-0.000010,-0.000004
299998,-0.000017,-0.000003,-0.000001,0.000004,-0.000005,-0.000004,-0.000006,-0.000018,-0.000015,0.000002,...,-0.000010,-0.000004,0.000000,0.000004,-0.000007,-0.000002,0.000004,-0.000011,-0.000009,-0.000003


In [31]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [32]:
model=XGBClassifier(n_estimators=500)
model.fit(x8,y8,early_stopping_rounds=10, eval_set=[(xv8, yv8)])
y_pred=model.predict(xt8)
print(classification_report(yt8,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.93145
[1]	validation_0-mlogloss:2.81771
[2]	validation_0-mlogloss:2.73923
[3]	validation_0-mlogloss:2.67700
[4]	validation_0-mlogloss:2.62570
[5]	validation_0-mlogloss:2.58917
[6]	validation_0-mlogloss:2.55329
[7]	validation_0-mlogloss:2.52323
[8]	validation_0-mlogloss:2.49181
[9]	validation_0-mlogloss:2.47261
[10]	validation_0-mlogloss:2.45166
[11]	validation_0-mlogloss:2.43362
[12]	validation_0-mlogloss:2.41513
[13]	validation_0-mlogloss:2.40333
[14]	validation_0-mlogloss:2.39216
[15]	validation_0-mlogloss:2.37805
[16]	validation_0-mlogloss:2.36761
[17]	validation_0-mlogloss:2.35508
[18]	validation_0-mlogloss:2.34854
[19]	validation_0-mlogloss:2.33602
[20]	validation_0-mlogloss:2.32327
[21]	validation_0-mlogloss:2.31347
[22]	validation_0-mlogloss:2.30482
[23]	validation_0-mlogloss:2.30002
[24]	validation_0-mlogloss:2.28978
[25]	validation_0-mlogloss:2.28668
[26]	validation_0-mlogloss:2.28195
[27]	validation_0-mlogloss:2.27670
[28]	validation_0-mlogloss:2.2

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10


9375/9375 [==============================] - 14s 1ms/step - loss: 1.9082 - val_loss: 2.0191
Epoch 2/10
9375/9375 [==============================] - 15s 2ms/step - loss: 1.3610 - val_loss: 2.1657
Epoch 3/10
9375/9375 [==============================] - 14s 1ms/step - loss: 1.2310 - val_loss: 2.2872
Epoch 4/10
9375/9375 [==============================] - 14s 1ms/step - loss: 1.1771 - val_loss: 2.2815
Epoch 5/10
9375/9375 [==============================] - 14s 1ms/step - loss: 1.1443 - val_loss: 2.4766
Epoch 6/10
9375/9375 [==============================] - 13s 1ms/step - loss: 1.1229 - val_loss: 2.4708
Epoch 7/10
9375/9375 [==============================] - 14s 1ms/step - loss: 1.1049 - val_loss: 2.4606
Epoch 8/10
9375/9375 [==============================] - 14s 1ms/step - loss: 1.0906 - val_loss: 2.4856
Epoch 9/10
9375/9375 [==============================] - 14s 1ms/step - loss: 1.0790 - val_loss: 2.6716
Epoch 10/10
9375/9375 [==============================] - 14s 1ms/step - loss: 1.0679

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

 114/1172 [=>............................] - ETA: 0s

1172/1172 [==============================] - 1s 867us/step
              precision    recall  f1-score   support

           0       0.29      0.47      0.36      1499
           1       0.33      0.62      0.43      1499
           2       0.40      0.57      0.47      1499
           3       0.49      0.39      0.44      1499
           4       0.68      0.78      0.72      1499
           5       0.51      0.71      0.60      1499
           6       0.60      0.67      0.64      1499
           7       0.59      0.45      0.51      1499
           8       0.46      0.70      0.55      1499
           9       0.66      0.71      0.69      1499
          10       0.22      0.15      0.18      1499
          11       0.23      0.17      0.20      1499
          12       0.18      0.12      0.15      1499
          13       0.20      0.25      0.22      1499
          14       0.26      0.15      0.19      1499
          15       0.31      0.35      0.33      1499
          16       0.7

In [17]:
import xgboost as xgb
D_train = xgb.DMatrix(x8, label=y8)
D_test = xgb.DMatrix(xt8, label=yt8)
model = xgb.train(D_train,num_class=25)
ypred=model.predict(xt8)
print(classification_report(yt8,ypred))

TypeError: train() got an unexpected keyword argument 'num_class'

## 0-16

In [33]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [34]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_10231/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_10231/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_10231/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.000000,0.000008,0.000001,0.000003,-0.000005,0.000001,-0.000003,-0.000008,-0.000004,0.000007,...,-0.000009,0.000000,0.000009,0.000017,0.000005,0.000002,0.000015,0.000003,0.000004,0.000012
299996,-0.000006,0.000014,0.000009,0.000012,0.000005,0.000010,0.000007,-0.000002,-0.000001,0.000012,...,-0.000005,0.000006,0.000013,0.000018,0.000007,0.000012,0.000015,0.000009,0.000012,0.000017
299997,0.000008,0.000017,0.000016,0.000021,0.000009,0.000010,0.000005,-0.000002,-0.000002,0.000014,...,-0.000012,-0.000003,0.000002,0.000004,-0.000008,-0.000005,0.000005,-0.000011,-0.000010,-0.000004
299998,-0.000017,-0.000003,-0.000001,0.000004,-0.000005,-0.000004,-0.000006,-0.000018,-0.000015,0.000002,...,-0.000010,-0.000004,0.000000,0.000004,-0.000007,-0.000002,0.000004,-0.000011,-0.000009,-0.000003


In [35]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [36]:
model2=XGBClassifier(n_estimators=500)
model2.fit(x16,y16,early_stopping_rounds=10, eval_set=[(xv16, yv16)])
y_pred=model2.predict(xt16)
print(classification_report(yt16,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.86806
[1]	validation_0-mlogloss:2.73679
[2]	validation_0-mlogloss:2.64125
[3]	validation_0-mlogloss:2.55989
[4]	validation_0-mlogloss:2.49798
[5]	validation_0-mlogloss:2.44695
[6]	validation_0-mlogloss:2.40216
[7]	validation_0-mlogloss:2.35693
[8]	validation_0-mlogloss:2.32183
[9]	validation_0-mlogloss:2.28848
[10]	validation_0-mlogloss:2.25645
[11]	validation_0-mlogloss:2.23106
[12]	validation_0-mlogloss:2.20833
[13]	validation_0-mlogloss:2.18272
[14]	validation_0-mlogloss:2.16264
[15]	validation_0-mlogloss:2.14244
[16]	validation_0-mlogloss:2.12256
[17]	validation_0-mlogloss:2.10579
[18]	validation_0-mlogloss:2.08784
[19]	validation_0-mlogloss:2.07081
[20]	validation_0-mlogloss:2.05692
[21]	validation_0-mlogloss:2.04071
[22]	validation_0-mlogloss:2.02457
[23]	validation_0-mlogloss:2.01073
[24]	validation_0-mlogloss:1.99799
[25]	validation_0-mlogloss:1.98731
[26]	validation_0-mlogloss:1.97624
[27]	validation_0-mlogloss:1.96748
[28]	validation_0-mlogloss:1.9

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10


9375/9375 [==============================] - 15s 1ms/step - loss: 1.3050 - val_loss: 1.8960
Epoch 2/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.6797 - val_loss: 1.9321
Epoch 3/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.5478 - val_loss: 2.1032
Epoch 4/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.4926 - val_loss: 2.2106
Epoch 5/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.4596 - val_loss: 2.4400
Epoch 6/10
9375/9375 [==============================] - 13s 1ms/step - loss: 0.4374 - val_loss: 2.3829
Epoch 7/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.4194 - val_loss: 2.7439
Epoch 8/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.4070 - val_loss: 2.6990
Epoch 9/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.3950 - val_loss: 2.5336
Epoch 10/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.3850

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

 177/1172 [===>..........................] - ETA: 0s

1172/1172 [==============================] - 1s 866us/step
              precision    recall  f1-score   support

           0       0.31      0.55      0.39      1499
           1       0.48      0.67      0.56      1499
           2       0.50      0.71      0.59      1499
           3       0.60      0.49      0.54      1499
           4       0.86      0.92      0.89      1499
           5       0.61      0.80      0.69      1499
           6       0.94      0.57      0.71      1499
           7       0.80      0.58      0.67      1499
           8       0.48      0.60      0.53      1499
           9       0.59      0.83      0.69      1499
          10       0.40      0.08      0.13      1499
          11       0.84      0.61      0.71      1499
          12       0.53      0.42      0.47      1499
          13       0.21      0.48      0.29      1499
          14       0.34      0.52      0.41      1499
          15       0.72      0.36      0.48      1499
          16       0.8

## 0-32

In [37]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [38]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_10231/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_10231/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_10231/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [39]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [40]:
model3=XGBClassifier(n_estimators=500)
model3.fit(x32,y32,early_stopping_rounds=10, eval_set=[(xv32, yv32)])
y_pred=model3.predict(xt32)
print(classification_report(yt32,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.70971
[1]	validation_0-mlogloss:2.51738
[2]	validation_0-mlogloss:2.38308
[3]	validation_0-mlogloss:2.27969
[4]	validation_0-mlogloss:2.19041
[5]	validation_0-mlogloss:2.11033
[6]	validation_0-mlogloss:2.04798
[7]	validation_0-mlogloss:1.99195
[8]	validation_0-mlogloss:1.94001
[9]	validation_0-mlogloss:1.88984
[10]	validation_0-mlogloss:1.84523
[11]	validation_0-mlogloss:1.80420
[12]	validation_0-mlogloss:1.76769
[13]	validation_0-mlogloss:1.73532
[14]	validation_0-mlogloss:1.70033
[15]	validation_0-mlogloss:1.67288
[16]	validation_0-mlogloss:1.65079
[17]	validation_0-mlogloss:1.62779
[18]	validation_0-mlogloss:1.60033
[19]	validation_0-mlogloss:1.57218
[20]	validation_0-mlogloss:1.55227
[21]	validation_0-mlogloss:1.53162
[22]	validation_0-mlogloss:1.51232
[23]	validation_0-mlogloss:1.49478
[24]	validation_0-mlogloss:1.48041
[25]	validation_0-mlogloss:1.46415
[26]	validation_0-mlogloss:1.44284
[27]	validation_0-mlogloss:1.42518
[28]	validation_0-mlogloss:1.4

In [34]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [35]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10


2023-10-04 11:33:03.291033: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 38400000 exceeds 10% of free system memory.


9375/9375 [==============================] - 15s 2ms/step - loss: 0.8175 - val_loss: 2.0148
Epoch 2/10
9375/9375 [==============================] - 14s 2ms/step - loss: 0.2687 - val_loss: 1.8710
Epoch 3/10
9375/9375 [==============================] - 14s 2ms/step - loss: 0.1857 - val_loss: 1.7827
Epoch 4/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.1519 - val_loss: 1.8325
Epoch 5/10
9375/9375 [==============================] - 14s 1ms/step - loss: 0.1314 - val_loss: 1.6044
Epoch 6/10
9375/9375 [==============================] - 14s 2ms/step - loss: 0.1172 - val_loss: 1.8141
Epoch 7/10
9375/9375 [==============================] - 15s 2ms/step - loss: 0.1065 - val_loss: 1.7100
Epoch 8/10
9375/9375 [==============================] - 14s 2ms/step - loss: 0.0997 - val_loss: 2.0032
Epoch 9/10
9375/9375 [==============================] - 15s 2ms/step - loss: 0.0931 - val_loss: 1.5126
Epoch 10/10
9375/9375 [==============================] - 14s 2ms/step - loss: 0.0864

In [36]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

 228/1172 [====>.........................] - ETA: 0s

1172/1172 [==============================] - 1s 880us/step
              precision    recall  f1-score   support

           0       0.81      0.59      0.68      1499
           1       0.75      0.88      0.81      1499
           2       0.67      0.81      0.74      1499
           3       0.57      0.56      0.56      1499
           4       0.94      0.97      0.96      1499
           5       0.90      0.74      0.81      1499
           6       0.91      0.77      0.84      1499
           7       0.85      0.63      0.72      1499
           8       0.53      0.94      0.68      1499
           9       0.85      0.88      0.87      1499
          10       0.65      0.61      0.63      1499
          11       0.98      0.72      0.83      1499
          12       0.64      0.86      0.74      1499
          13       0.38      0.82      0.52      1499
          14       0.54      0.74      0.62      1499
          15       0.69      0.45      0.55      1499
          16       0.9

## 0-64

In [41]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [42]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [43]:
model4=XGBClassifier(n_estimators=500)
model4.fit(x,y,early_stopping_rounds=10, eval_set=[(xv, yv)])
y_pred=model4.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:2.44111
[1]	validation_0-mlogloss:2.20961
[2]	validation_0-mlogloss:2.03998
[3]	validation_0-mlogloss:1.91108
[4]	validation_0-mlogloss:1.80742
[5]	validation_0-mlogloss:1.71138
[6]	validation_0-mlogloss:1.63310
[7]	validation_0-mlogloss:1.56635
[8]	validation_0-mlogloss:1.50783
[9]	validation_0-mlogloss:1.45546
[10]	validation_0-mlogloss:1.40198
[11]	validation_0-mlogloss:1.35729
[12]	validation_0-mlogloss:1.32107
[13]	validation_0-mlogloss:1.28236
[14]	validation_0-mlogloss:1.24948
[15]	validation_0-mlogloss:1.21666
[16]	validation_0-mlogloss:1.18832
[17]	validation_0-mlogloss:1.15846
[18]	validation_0-mlogloss:1.13277
[19]	validation_0-mlogloss:1.10898
[20]	validation_0-mlogloss:1.08729
[21]	validation_0-mlogloss:1.06274
[22]	validation_0-mlogloss:1.04293
[23]	validation_0-mlogloss:1.02324
[24]	validation_0-mlogloss:1.00372
[25]	validation_0-mlogloss:0.98551
[26]	validation_0-mlogloss:0.97020
[27]	validation_0-mlogloss:0.95367
[28]	validation_0-mlogloss:0.9

In [39]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(25, activation = 'linear', name = "L6"),
    ]
)

In [40]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10


2023-10-04 11:35:29.030629: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76800000 exceeds 10% of free system memory.


9375/9375 [==============================] - 15s 2ms/step - loss: 0.5260 - val_loss: 1.2493
Epoch 2/10
9375/9375 [==============================] - 14s 2ms/step - loss: 0.1637 - val_loss: 1.1901
Epoch 3/10
9375/9375 [==============================] - 14s 2ms/step - loss: 0.1160 - val_loss: 1.1938
Epoch 4/10
9375/9375 [==============================] - 14s 2ms/step - loss: 0.0946 - val_loss: 1.1484
Epoch 5/10
9375/9375 [==============================] - 14s 2ms/step - loss: 0.0795 - val_loss: 1.0209
Epoch 6/10
9375/9375 [==============================] - 14s 2ms/step - loss: 0.0712 - val_loss: 0.8775
Epoch 7/10
9375/9375 [==============================] - 15s 2ms/step - loss: 0.0634 - val_loss: 0.9754
Epoch 8/10
9375/9375 [==============================] - 15s 2ms/step - loss: 0.0588 - val_loss: 1.0735
Epoch 9/10
9375/9375 [==============================] - 15s 2ms/step - loss: 0.0556 - val_loss: 1.0181
Epoch 10/10
9375/9375 [==============================] - 15s 2ms/step - loss: 0.0509

In [41]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

 117/1172 [=>............................] - ETA: 0s

1172/1172 [==============================] - 1s 880us/step
              precision    recall  f1-score   support

           0       0.81      0.84      0.83      1499
           1       0.92      0.94      0.93      1499
           2       0.71      0.66      0.68      1499
           3       0.85      0.83      0.84      1499
           4       0.98      1.00      0.99      1499
           5       0.86      0.91      0.89      1499
           6       0.96      0.97      0.97      1499
           7       0.75      0.79      0.77      1499
           8       0.82      0.91      0.87      1499
           9       0.89      0.99      0.94      1499
          10       0.88      0.33      0.48      1499
          11       0.87      0.92      0.89      1499
          12       0.67      0.95      0.79      1499
          13       0.41      0.86      0.56      1499
          14       0.73      0.83      0.77      1499
          15       0.94      0.41      0.57      1499
          16       0.9